In [13]:
from faster_rcnn import config
from faster_rcnn import vgg
from faster_rcnn import parser
from faster_rcnn import data_generators
from faster_rcnn import losses
from faster_rcnn import roi_helpers

import pickle
import time
import numpy as np
from imp import reload

from keras.utils import generic_utils
from keras.optimizers import Adam
from keras.layers import Input
from keras.models import Model

In [14]:
# 用于修改模块后重新加载
reload(config)
reload(vgg)
reload(parser)
reload(data_generators)
reload(losses)
reload(roi_helpers)

<module 'faster_rcnn.roi_helpers' from 'C:\\Users\\paops\\Desktop\\sqdt\\vehicledetector\\faster_rcnn\\roi_helpers.py'>

In [15]:
C = config.Config() # 参数
# 在训练时进行数据增广
C.use_horizontal_flips = True
C.use_vertical_flips = True
C.rot_90 = True

img_path = 'data_object_image_2/training/image_2' # 训练图片路径
label_path = 'data_object_label_2/training/label_2' # 训练标签路径

all_imgs, classes_count, class_mapping = parser.get_data(img_path, label_path)
C.class_mapping = class_mapping

# 保存训练参数
with open(C.config_filename, 'wb') as config_f:
    pickle.dump(C, config_f)

train_imgs = [s for s in all_imgs if s['imageset'] == 'trainval']
val_imgs = [s for s in all_imgs if s['imageset'] == 'test']

print('Num train samples {}'.format(len(train_imgs)))
print('Num val samples {}'.format(len(val_imgs)))

# 产生训练与验证数据
data_gen_train = data_generators.get_anchor_gt(train_imgs, classes_count, img_path, C, mode = 'train')
data_gen_val = data_generators.get_anchor_gt(val_imgs, classes_count, img_path, C, mode = 'val')

Num train samples 824
Num val samples 176


In [16]:
classes_count

{'people': 865, 'vehicle': 4334, 'bg': 0}

In [17]:
class_mapping

{'people': 0, 'vehicle': 1, 'bg': 2}

In [18]:
all_imgs[0]

{'filepath': '000018.png',
 'width': 1242,
 'height': 375,
 'bboxes': [{'class': 'vehicle', 'x1': 523, 'x2': 550, 'y1': 194, 'y2': 219},
  {'class': 'vehicle', 'x1': 572, 'x2': 606, 'y1': 185, 'y2': 219},
  {'class': 'vehicle', 'x1': 387, 'x2': 473, 'y1': 166, 'y2': 213}],
 'imageset': 'trainval'}

In [19]:
# 定义输入维度
img_input = Input(shape = (None, None, 3))
roi_input = Input(shape = (None, 4))

# 定义共享层
shared_layers = vgg.nn_base(img_input, trainable = False)

# 定义共享层基础上的rpn层
num_anchors = len(C.anchor_box_scales) * len(C.anchor_box_ratios)
rpn = vgg.rpn(shared_layers, num_anchors)

# 定义共享层及rpn层基础上的分类层
classifier = vgg.classifier(shared_layers, roi_input, C.num_rois, nb_classes = len(classes_count), trainable = True)

model_rpn = Model(img_input, rpn[:2])
model_classifier = Model([img_input, roi_input], classifier)
model_all = Model([img_input, roi_input], rpn[:2] + classifier) # 该模型只用于保存训练权重

# 依据层名载入预训练参数
model_rpn.load_weights(C.base_net_weights, by_name = True)
model_classifier.load_weights(C.base_net_weights, by_name = True)

# 模型编译
optimizer = Adam(lr = 1e-3)
optimizer_classifier = Adam(lr = 1e-3)
model_rpn.compile(optimizer = optimizer, loss = [losses.rpn_loss_cls(num_anchors), losses.rpn_loss_regr(num_anchors)])
model_classifier.compile(optimizer = optimizer_classifier, loss = [losses.class_loss_cls, losses.class_loss_regr(len(classes_count) - 1)], metrics={'dense_class_{}'.format(len(classes_count)): 'accuracy'})
model_all.compile(optimizer = 'sgd', loss = 'mae')

In [20]:
# 设置每个epoch长度，以及epoch数
epoch_length = 500
num_epochs = 10
iter_num = 0

losses = np.zeros((epoch_length, 5))
rpn_accuracy_rpn_monitor = []
rpn_accuracy_for_epoch = []
start_time = time.time()
best_loss = np.Inf

print('Starting training')

for epoch_num in range(num_epochs):
    progbar = generic_utils.Progbar(epoch_length)
    print('Epoch {}/{}'.format(epoch_num + 1, num_epochs))
    while 1:
        try:
            # 每到epoch长度的迭代数时输出与bounding box重合的平均数
            if len(rpn_accuracy_rpn_monitor) == epoch_length:
                mean_overlapping_bboxes = sum(rpn_accuracy_rpn_monitor) / len(rpn_accuracy_rpn_monitor)
                rpn_accuracy_rpn_monitor = []
                print('Average number of overlapping bounding boxes from RPN = {} for {} previous iterations'.format(mean_overlapping_bboxes, epoch_length))
                if mean_overlapping_bboxes == 0:
                    print('RPN is not producing bounding boxes that overlap the ground truth boxes. Check RPN settings or keep training.')
            # 产生训练特征，训练标签，与图像数据
            X, Y, img_data = next(data_gen_train)
            # 训练rpn网络
            loss_rpn = model_rpn.train_on_batch(X, Y)
            # 利用训练的rpn网络进行预测
            P_rpn = model_rpn.predict_on_batch(X)
            # 计算用于分类的bounding boxes、类别独热码、与应类别标签及坐标
            R = roi_helpers.rpn_to_roi(P_rpn[0], P_rpn[1], C, use_regr = True, overlap_thresh = 0.7, max_boxes = 300)
            X2, Y1, Y2 = roi_helpers.calc_iou(R, img_data, C, class_mapping)
            
            if X2 is None:
                rpn_accuracy_rpn_monitor.append(0)
                rpn_accuracy_for_epoch.append(0)
                continue
            
            # 背景为1的样本为neg样本，背景为0的样本为pos样本
            neg_samples = np.where(Y1[0, :, -1] == 1)
            pos_samples = np.where(Y1[0, :, -1] == 0)
            
            if len(neg_samples) > 0:
                neg_samples = neg_samples[0]
            else:
                neg_samples = []

            if len(pos_samples) > 0:
                pos_samples = pos_samples[0]
            else:
                pos_samples = []

            rpn_accuracy_rpn_monitor.append(len(pos_samples))
            rpn_accuracy_for_epoch.append((len(pos_samples)))
            
            # 若pos样本数量小于num_rois的一半，则选取所有的pos样本，否则，选取num_rois一半数量的pos样本，剩下的样本使用neg样本
            if C.num_rois > 1:
                if len(pos_samples) < C.num_rois // 2:
                    selected_pos_samples = pos_samples.tolist()
                else:
                    selected_pos_samples = np.random.choice(pos_samples, C.num_rois // 2, replace = False).tolist()
                try:
                    selected_neg_samples = np.random.choice(neg_samples, C.num_rois - len(selected_pos_samples), replace = False).tolist()
                except:
                    selected_neg_samples = np.random.choice(neg_samples, C.num_rois - len(selected_pos_samples), replace = True).tolist()

                sel_samples = selected_pos_samples + selected_neg_samples
            
            # 若num_rois为1时，随机选取pos或neg样本
            else:
                selected_pos_samples = pos_samples.tolist()
                selected_neg_samples = neg_samples.tolist()
                if np.random.randint(0, 2):
                    sel_samples = random.choice(neg_samples)
                else:
                    sel_samples = random.choice(pos_samples)

            # 训练分类模型
            loss_class = model_classifier.train_on_batch([X, X2[:, sel_samples, :]], [Y1[:, sel_samples, :], Y2[:, sel_samples, :]])

            losses[iter_num, 0] = loss_rpn[1] # rpn分类损失
            losses[iter_num, 1] = loss_rpn[2] # rpn回归损失

            losses[iter_num, 2] = loss_class[1] # classifier分类损失
            losses[iter_num, 3] = loss_class[2] # classifier回归损失
            losses[iter_num, 4] = loss_class[3] # classifier分类准确率

            iter_num += 1
            
            # 实时反馈模型损失
            progbar.update(iter_num, [('rpn_cls', np.mean(losses[:iter_num, 0])), ('rpn_regr', np.mean(losses[:iter_num, 1])),
                                      ('detector_cls', np.mean(losses[:iter_num, 2])), ('detector_regr', np.mean(losses[:iter_num, 3]))])
            
            if iter_num == epoch_length:
                loss_rpn_cls = np.mean(losses[:, 0])
                loss_rpn_regr = np.mean(losses[:, 1])
                loss_class_cls = np.mean(losses[:, 2])
                loss_class_regr = np.mean(losses[:, 3])
                class_acc = np.mean(losses[:, 4])

                mean_overlapping_bboxes = sum(rpn_accuracy_for_epoch) / len(rpn_accuracy_for_epoch)
                rpn_accuracy_for_epoch = []

                print('Mean number of bounding boxes from RPN overlapping ground truth boxes: {}'.format(mean_overlapping_bboxes))
                print('Classifier accuracy for bounding boxes from RPN: {}'.format(class_acc))
                print('Loss RPN classifier: {}'.format(loss_rpn_cls))
                print('Loss RPN regression: {}'.format(loss_rpn_regr))
                print('Loss Detector classifier: {}'.format(loss_class_cls))
                print('Loss Detector regression: {}'.format(loss_class_regr))
                print('Elapsed time: {}'.format(time.time() - start_time))

                curr_loss = loss_rpn_cls + loss_rpn_regr + loss_class_cls + loss_class_regr
                iter_num = 0
                start_time = time.time()

                # 当前损失小于最佳损失时，保存当前损失与模型参数
                if curr_loss < best_loss:
                    print('Total loss decreased from {} to {}, saving weights'.format(best_loss,curr_loss))
                    best_loss = curr_loss
                    model_all.save_weights(C.model_path)

                break

        except Exception as e:
            print('Exception: {}'.format(e))
            continue

print('Training complete, exiting.')

Starting training
Epoch 1/10
500/500 [==============================] - 5089s 10s/step - rpn_cls: 4.5171 - rpn_regr: 1.3767 - detector_cls: 2.6282 - detector_regr: 3.0142
Mean number of bounding boxes from RPN overlapping ground truth boxes: 2.828
Classifier accuracy for bounding boxes from RPN: 0.71175
Loss RPN classifier: 3.8614743045460402
Loss RPN regression: 1.2697716426253318
Loss Detector classifier: 2.472912635830823
Loss Detector regression: 2.21471747019887
Elapsed time: 5088.822981595993
Total loss decreased from inf to 9.818876053201066, saving weights
Epoch 2/10
Average number of overlapping bounding boxes from RPN = 2.828 for 500 previous iterations
500/500 [==============================] - 5334s 11s/step - rpn_cls: 3.0578 - rpn_regr: 1.1157 - detector_cls: 1.4495 - detector_regr: 1.0205
Mean number of bounding boxes from RPN overlapping ground truth boxes: 4.222
Classifier accuracy for bounding boxes from RPN: 0.6455
Loss RPN classifier: 3.0095149977416504
Loss RPN regr